In [1]:
#Import Statements
import pandas as pd

import xml.etree.ElementTree as ET # documentation: https://docs.python.org/3/library/xml.etree.elementtree.html
import numpy as np
import glob
import os

In [2]:
# define path variables to use for factor arrays and output paths
path = '/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/'
addon_path = path + 'input/addons/ensemble_addons/'
write_path = path + 'exe/config_ensemble_joo/'
outputs_path = path + '/output/db_ensemble_joo/'

In [3]:
water_file_path = '/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/input/addons/ensemble_addons/water'

scen_list_water_ssp126 = os.listdir(water_file_path+'/ssp126')
scen_list_water_ssp370 = os.listdir(water_file_path+'/ssp370')

scen_list_water_ssp126 = [scen_list_water_ssp126.replace('.xml', '') for scen_list_water_ssp126 in scen_list_water_ssp126]
scen_list_water_ssp370 = [scen_list_water_ssp370.replace('.xml', '') for scen_list_water_ssp370 in scen_list_water_ssp370]

In [4]:
len(scen_list_water_ssp370) #Should be 6, 3 GCM * Res/Exp

6

# Generate Configuration XML files (3888 = 1296 * 3)

## 1) Reference Emissions (1296 Scenarios): Couple with SSP370

In [18]:
# using xml.etree.ElementTree, read in base configuration file, parse, and find ScenarioComponents child
tree=ET.parse('Base_Configuration/configuration_base_Reference.xml')
root=tree.getroot() 
scenComp=root.find('ScenarioComponents')

In [19]:
# print out xml contents for quick viewing
for child in root:
    for i in range(len(child)):
        print(child.tag, child[i].attrib, child[i].text)
    print('')

Files {'name': 'xmlInputFileName'} ../input/gcamdata/xml/modeltime.xml
Files {'name': 'BatchFileName'} batch_ag.xml
Files {'name': 'policy-target-file'} ../input/policy/forcing_target_4p5.xml
Files {'name': 'GHGInputFileName'} ../input/magicc/inputs/input_gases.emk
Files {'write-output': '1', 'append-scenario-name': '0', 'name': 'xmldb-location'} ../output/db_ensemble_joo/Reference
Files {'write-output': '1', 'append-scenario-name': '0', 'name': 'restart'} ./restart/restart
Files {'write-output': '1', 'append-scenario-name': '1', 'name': 'xmlDebugFileName'} debug.xml
Files {'write-output': '1', 'append-scenario-name': '0', 'name': 'climatFileName'} gas.emk
Files {'write-output': '1', 'append-scenario-name': '1', 'name': 'costCurvesOutputFileName'} cost_curves.xml
Files {'write-output': '1', 'append-scenario-name': '0', 'name': 'batchCSVOutputFile'} batch-csv-out.csv
Files {'write-output': '0', 'append-scenario-name': '0', 'name': 'supplyDemandOutputFileName'} SDCurves.csv
Files {'write

In [23]:
tree=ET.parse('Base_Configuration/configuration_base_Reference.xml')
root=tree.getroot() 
scenComp=root.find('ScenarioComponents')

ag_prd = scenComp.find(".//*[@name='ag_prodchange']")
ref_text_ag_prd = ag_prd.text

fd = scenComp.find(".//*[@name='demand']")
ref_text_fdem = fd.text

hddcdd_ccimp = scenComp.find(".//*[@name='hddcdd_ccimp']")
ref_text_hddcdd_ccimp = hddcdd_ccimp.text

ag_trade = scenComp.find(".//*[@name='ag_trade']")
ref_text_ag_trade = ag_trade.text

In [24]:
write_path = '/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ref/'

In [25]:
for socio in np.array([2,3,5]):
    for agprd in np.array([1,2,3]):
        for fdem in np.array([0,1]):
            for bld in np.array(['Ref','BEE','SatLevel','BEESatLevel']):
                for gcm in np.array(['canesm5','gfdl','ipsl']):
                    for res in np.array([0,1]):
                        for incdist in np.array([2,3,5]):
                    
                            # Update Scenario Name (config_SGP2_SI2_ELC0_A0_F0_RB0_E0_G1_RCP2_RES0)
                            scen ='kim_mcfe_ref_'
                            root[2][0].text=scen

                            ### 1. Socioeconomics: Population & GDP #############
                            soc = ET.SubElement(scenComp,'Value')
                            soc.text='../input/gcamdata/xml/socioeconomics_gSSP{}.xml'.format(str(socio))    
                            soc.attrib = {'name':'socioeconomics'}
                            scen += 'soc{}'.format(str(socio))

                            ### 2. Agricultural Productivity #############
                            ag_prd.text='../input/gcamdata/xml/ag_prodchange_ssp{}_IRR_MGMT.xml'.format(str(agprd))    
                            scen += '_ag{}'.format(str(agprd))

                            ### 3. Food Demand Sensitivity ############                      
                            if  fdem > 0:
                                fd.text='../input/addons/ensemble_addons/food/ag_an_demand_input_multiple_consumers_Low.xml'
                            else:
                                fd.text='../input/addons/ensemble_addons/food/ag_an_demand_input_multiple_consumers_NewRef.xml'
                            scen += '_fd{}'.format(str(fdem))

                            ### 4. Building Energy Efficiency & Demand Sensitivity ############
                            bed = ET.SubElement(scenComp,'Value')
                            bed.text='../input/addons/ensemble_addons/energy/building_det_{}.xml'.format(str(bld))    
                            bed.attrib = {'name':'building_demand_addon'}
                            scen += '_bld{}'.format(str(bld))

                            
                            ### 5-1. GCM: Runoff-Water ############
                            ro = ET.SubElement(scenComp,'Value')
                            if res > 0:
                                ro.text='../input/addons/ensemble_addons/water/ssp370/{}_ssp370_exp.xml'.format(str(gcm))    
                            else:
                                ro.text='../input/addons/ensemble_addons/water/ssp370/{}_ssp370_res.xml'.format(str(gcm))
                            ro.attrib = {'name':'climate_runoff'}
                            scen += '_{}'.format(str(gcm)+'_res'+str(res))       

                            ### 5-2. GCM: HDD/CDD ############
                            hddcdd_ccimp.text='../input/addons/ensemble_addons/energy/hddcdd/hdcd_{}_ssp370.xml'.format(str(gcm))    
                            
                            ### 6. Income Distributions for Energy/Food ##########
                            ind_fd = ET.SubElement(scenComp,'Value')
                            ind_fd.text='../input/addons/ensemble_addons/soc/food_inc_share_ssp{}.xml'.format(str(incdist))    
                            ind_fd.attrib = {'name':'income_dist_food'}

                            ind_en = ET.SubElement(scenComp,'Value')
                            ind_en.text='../input/addons/ensemble_addons/soc/energy_inc_share_ssp{}.xml'.format(str(incdist))    
                            ind_en.attrib = {'name':'income_dist_energy'}

                            scen += '_icd{}'.format(str(incdist))                        

                            ### 7. Level of Agricultural Trade ############
                            ag_trade.text='../input/addons/ensemble_addons/food/trade/ag_trade_sw_ssp{}.xml'.format(str(socio))    

                            # Final: xmldb-location within Files element
                            root[0][4].text='../output/db_ensemble_joo/ref/' + scen
                            root[0][6].attrib={'write-output': '0', 'append-scenario-name': '0', 'name': 'xmlDebugFileName'}

                            root[2][0].text=scen  

                            tree.write(write_path +'soc{}'.format(str(socio))  +'/' +  scen + '.xml')

                            scenComp.remove(soc)                        
                            scenComp.remove(bed)
                            scenComp.remove(ro)
                            scenComp.remove(ind_fd)
                            scenComp.remove(ind_en)

                            ag_prd.text = ref_text_ag_prd
                            fd.text = ref_text_fdem
                            hddcdd_ccimp.text = ref_text_hddcdd_ccimp
                            ag_trade.text = ref_text_ag_trade

## 2) CO2 = EP (1296 Scenarios): Couple with SSP126

In [11]:
# using xml.etree.ElementTree, read in base configuration file, parse, and find ScenarioComponents child
tree=ET.parse('Base_Configuration/configuration_base_EP_4NEB_noDAC.xml')
root=tree.getroot() 
scenComp=root.find('ScenarioComponents')

In [12]:
write_path = '/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndc/'

In [13]:
ag_prd = scenComp.find(".//*[@name='ag_prodchange']")
ref_text_ag_prd = ag_prd.text

fd = scenComp.find(".//*[@name='demand']")
ref_text_fdem = fd.text

hddcdd_ccimp = scenComp.find(".//*[@name='hddcdd_ccimp']")
ref_text_hddcdd_ccimp = hddcdd_ccimp.text

ag_trade = scenComp.find(".//*[@name='ag_trade']")
ref_text_ag_trade = ag_trade.text

In [14]:
for socio in np.array([2,3,5]):
    for agprd in np.array([1,2,3]):
        for fdem in np.array([0,1]):
            for bld in np.array(['Ref','BEE','SatLevel','BEESatLevel']):
                for gcm in np.array(['canesm5','gfdl','ipsl']):
                    for res in np.array([0,1]):
                        for incdist in np.array([2,3,5]):
                    
                            # Update Scenario Name (config_SGP2_SI2_ELC0_A0_F0_RB0_E0_G1_RCP2_RES0)
                            scen ='kim_mcfe_ndc_'
                            root[2][0].text=scen

                            ### 1. Socioeconomics: Population & GDP #############
                            soc = ET.SubElement(scenComp,'Value')
                            soc.text='../input/gcamdata/xml/socioeconomics_gSSP{}.xml'.format(str(socio))    
                            soc.attrib = {'name':'socioeconomics'}
                            scen += 'soc{}'.format(str(socio))

                            ### 2. Agricultural Productivity #############
                            ag_prd.text='../input/gcamdata/xml/ag_prodchange_ssp{}_IRR_MGMT.xml'.format(str(agprd))    
                            scen += '_ag{}'.format(str(agprd))

                            ### 3. Food Demand Sensitivity ############                      
                            if  fdem > 0:
                                fd.text='../input/addons/ensemble_addons/food/ag_an_demand_input_multiple_consumers_Low.xml'
                            else:
                                fd.text='../input/addons/ensemble_addons/food/ag_an_demand_input_multiple_consumers_NewRef.xml'
                            scen += '_fd{}'.format(str(fdem))

                            ### 4. Building Energy Efficiency & Demand Sensitivity ############
                            bed = ET.SubElement(scenComp,'Value')
                            bed.text='../input/addons/ensemble_addons/energy/building_det_{}.xml'.format(str(bld))    
                            bed.attrib = {'name':'building_demand_addon'}
                            scen += '_bld{}'.format(str(bld))

                            
                            ### 5-1. GCM: Runoff-Water ############
                            ro = ET.SubElement(scenComp,'Value')
                            if res > 0:
                                ro.text='../input/addons/ensemble_addons/water/ssp126/{}_ssp126_exp.xml'.format(str(gcm))    
                            else:
                                ro.text='../input/addons/ensemble_addons/water/ssp126/{}_ssp126_res.xml'.format(str(gcm))
                            ro.attrib = {'name':'climate_runoff'}
                            scen += '_{}'.format(str(gcm)+'_res'+str(res))       

                            ### 5-2. GCM: HDD/CDD ############
                            hddcdd_ccimp.text='../input/addons/ensemble_addons/energy/hddcdd/hdcd_{}_ssp126.xml'.format(str(gcm))    
                            
                            ### 6. Income Distributions for Energy/Food ##########
                            ind_fd = ET.SubElement(scenComp,'Value')
                            ind_fd.text='../input/addons/ensemble_addons/soc/food_inc_share_ssp{}.xml'.format(str(incdist))    
                            ind_fd.attrib = {'name':'income_dist_food'}

                            ind_en = ET.SubElement(scenComp,'Value')
                            ind_en.text='../input/addons/ensemble_addons/soc/energy_inc_share_ssp{}.xml'.format(str(incdist))    
                            ind_en.attrib = {'name':'income_dist_energy'}

                            scen += '_icd{}'.format(str(incdist))                        

                            ### 7. Level of Agricultural Trade ############
                            ag_trade.text='../input/addons/ensemble_addons/food/trade/ag_trade_sw_ssp{}.xml'.format(str(socio))    

                            # Final: xmldb-location within Files element
                            root[0][4].text='../output/db_ensemble_joo/ndc/' + scen
                            root[0][6].attrib={'write-output': '0', 'append-scenario-name': '0', 'name': 'xmlDebugFileName'}

                            root[2][0].text=scen  

                            tree.write(write_path +'soc{}'.format(str(socio))  +'/' +  scen + '.xml')

                            scenComp.remove(soc)                        
                            scenComp.remove(bed)
                            scenComp.remove(ro)
                            scenComp.remove(ind_fd)
                            scenComp.remove(ind_en)

                            ag_prd.text = ref_text_ag_prd
                            fd.text = ref_text_fdem
                            hddcdd_ccimp.text = ref_text_hddcdd_ccimp
                            ag_trade.text = ref_text_ag_trade

## 3) CO2 = EP + Land Conservation (1296 Scenarios): Couple with SSP126

In [5]:
# using xml.etree.ElementTree, read in base configuration file, parse, and find ScenarioComponents child
tree=ET.parse('Base_Configuration/configuration_base_EP_4NEB_noDAC_LC.xml')
root=tree.getroot() 
scenComp=root.find('ScenarioComponents')

In [6]:
write_path = '/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndl/'


In [7]:
ag_prd = scenComp.find(".//*[@name='ag_prodchange']")
ref_text_ag_prd = ag_prd.text

fd = scenComp.find(".//*[@name='demand']")
ref_text_fdem = fd.text

hddcdd_ccimp = scenComp.find(".//*[@name='hddcdd_ccimp']")
ref_text_hddcdd_ccimp = hddcdd_ccimp.text

ag_trade = scenComp.find(".//*[@name='ag_trade']")
ref_text_ag_trade = ag_trade.text

In [8]:
for socio in np.array([2,3,5]):
    for agprd in np.array([1,2,3]):
        for fdem in np.array([0,1]):
            for bld in np.array(['Ref','BEE','SatLevel','BEESatLevel']):
                for gcm in np.array(['canesm5','gfdl','ipsl']):
                    for res in np.array([0,1]):
                        for incdist in np.array([2,3,5]):
                            
                            # Update Scenario Name (config_SGP2_SI2_ELC0_A0_F0_RB0_E0_G1_RCP2_RES0)
                            scen ='kim_mcfe_ndl_'
                            root[2][0].text=scen

                            ### 1. Socioeconomics: Population & GDP #############
                            soc = ET.SubElement(scenComp,'Value')
                            soc.text='../input/gcamdata/xml_landcons/socioeconomics_gSSP{}.xml'.format(str(socio))    
                            soc.attrib = {'name':'socioeconomics'}
                            scen += 'soc{}'.format(str(socio))

                            ### 2. Agricultural Productivity #############
                            ag_prd.text='../input/gcamdata/xml_landcons/ag_prodchange_ssp{}_IRR_MGMT.xml'.format(str(agprd))    
                            scen += '_ag{}'.format(str(agprd))

                            ### 3. Food Demand Sensitivity ############                      
                            if  fdem > 0:
                                fd.text='../input/addons/ensemble_addons/food/ag_an_demand_input_multiple_consumers_Low.xml'
                            else:
                                fd.text='../input/addons/ensemble_addons/food/ag_an_demand_input_multiple_consumers_NewRef.xml'
                            scen += '_fd{}'.format(str(fdem))

                            ### 4. Building Energy Efficiency & Demand Sensitivity ############
                            bed = ET.SubElement(scenComp,'Value')
                            bed.text='../input/addons/ensemble_addons/energy/building_det_{}.xml'.format(str(bld))    
                            bed.attrib = {'name':'building_demand_addon'}
                            scen += '_bld{}'.format(str(bld))

                            
                            ### 5-1. GCM: Runoff-Water ############
                            ro = ET.SubElement(scenComp,'Value')
                            if res > 0:
                                ro.text='../input/addons/ensemble_addons/water/ssp126/{}_ssp126_exp.xml'.format(str(gcm))    
                            else:
                                ro.text='../input/addons/ensemble_addons/water/ssp126/{}_ssp126_res.xml'.format(str(gcm))
                            ro.attrib = {'name':'climate_runoff'}
                            scen += '_{}'.format(str(gcm)+'_res'+str(res))       

                            ### 5-2. GCM: HDD/CDD ############
                            hddcdd_ccimp.text='../input/addons/ensemble_addons/energy/hddcdd/hdcd_{}_ssp126.xml'.format(str(gcm))    
                            
                            ### 6. Income Distributions for Energy/Food ##########
                            ind_fd = ET.SubElement(scenComp,'Value')
                            ind_fd.text='../input/addons/ensemble_addons/soc/food_inc_share_ssp{}.xml'.format(str(incdist))    
                            ind_fd.attrib = {'name':'income_dist_food'}

                            ind_en = ET.SubElement(scenComp,'Value')
                            ind_en.text='../input/addons/ensemble_addons/soc/energy_inc_share_ssp{}.xml'.format(str(incdist))    
                            ind_en.attrib = {'name':'income_dist_energy'}

                            scen += '_icd{}'.format(str(incdist))                        

                            ### 7. Level of Agricultural Trade ############
                            ag_trade.text='../input/addons/ensemble_addons/food/trade/ag_trade_sw_ssp{}.xml'.format(str(socio))    

                            # Final: xmldb-location within Files element
                            root[0][4].text='../output/db_ensemble_joo/ndl/' + scen
                            root[0][6].attrib={'write-output': '0', 'append-scenario-name': '0', 'name': 'xmlDebugFileName'}

                            root[2][0].text=scen  

                            tree.write(write_path +'soc{}'.format(str(socio))  +'/' +  scen + '.xml')

                            scenComp.remove(soc)                        
                            scenComp.remove(bed)
                            scenComp.remove(ro)
                            scenComp.remove(ind_fd)
                            scenComp.remove(ind_en)

                            ag_prd.text = ref_text_ag_prd
                            fd.text = ref_text_fdem
                            hddcdd_ccimp.text = ref_text_hddcdd_ccimp
                            ag_trade.text = ref_text_ag_trade

In [72]:
ref_soc2 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ref/soc2/')
ref_soc3 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ref/soc3/')
ref_soc5 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ref/soc5/')

Ref_Sce_List = pd.concat([pd.DataFrame(ref_soc2), pd.DataFrame(ref_soc3), pd.DataFrame(ref_soc5)]).reset_index(drop=True)

In [73]:
ndc_soc2 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndc/soc2/')
ndc_soc3 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndc/soc3/')
ndc_soc5 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndc/soc5/')

NDC_Sce_List = pd.concat([pd.DataFrame(ndc_soc2), pd.DataFrame(ndc_soc3), pd.DataFrame(ndc_soc5)]).reset_index(drop=True)

In [74]:
ndl_soc2 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndl/soc2/')
ndl_soc3 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndl/soc3/')
ndl_soc5 = os.listdir('/cluster/tufts/lamontagnelab/gkim14/GCAM/mult_cons_food_plus_energy/gcam-core/exe/config_ensemble_joo/ndl/soc5/')

NDL_Sce_List = pd.concat([pd.DataFrame(ndl_soc2), pd.DataFrame(ndl_soc3), pd.DataFrame(ndl_soc5)]).reset_index(drop=True)

In [75]:
list_concat = pd.concat([Ref_Sce_List, NDC_Sce_List, NDL_Sce_List], axis=1)
list_concat.columns=["Ref","NDC",'NDL']

list_concat['Ref'] = list_concat['Ref'].str.replace('.xml','')
list_concat['NDC'] = list_concat['NDC'].str.replace('.xml','')
list_concat['NDL'] = list_concat['NDL'].str.replace('.xml','')

In [77]:
sdir = '/cluster/tufts/lamontagnelab/gkim14/MSD_Scenarios/Jupyter_Notebooks/Joo_database_MCFE_ensemble/Result_Analysis/Ensemble_Runs_Quality_Check'
list_concat.to_csv(sdir + '/sce_full_list.csv')